# Imports and Paths

In [157]:
from urllib import request
from bs4 import BeautifulSoup, Comment, Tag
import pandas as pd
import re

In [379]:
PATH = '/home/msnow/git/fantasy_football/data/external/game_data/2016/'

In [381]:
bs_pg = 'https://www.pro-football-reference.com/years/'
yr = 2016
pg = request.urlopen(f'{bs_pg}{yr}')
soup = BeautifulSoup(pg, 'html.parser')
with open(f"{PATH}{yr}.html", "w") as file:
    file.write(str(soup))

In [364]:
comments = soup.find_all(string=lambda text: isinstance(text, Comment))
table = BeautifulSoup(str(comments), 'lxml')

In [375]:
# th_rows = table.find_all('th')
tip_dict = {}
for i in th_rows:
    if 'data-tip' in i.attrs:
        tip_dict[i['data-stat']] = i['data-tip']

In [376]:
tip_dict

{'week_num': 'Week number in season',
 'pts_win': 'Points Scored by the winning team (first one listed)',
 'pts_lose': 'Points Scored by the losing team (second one listed)',
 'wins': 'Games Won',
 'losses': 'Games Lost',
 'ties': 'Tie Games',
 'reason': 'Reason the team is seeded the way it is, based on NFL tiebreaker rules',
 'ranker': '<b>Rank</b><br>This is a count of the rows from top to bottom.<br>It is recalculated following the sorting of a column.',
 'g': 'Games played',
 'points': 'Points Scored by team',
 'plays_offense': 'Offensive Plays: Pass Attempts + Rush Attempts + Times Sacked',
 'yds_per_play_offense': 'Yards per Offensive Play<br>(Rush + Pass Yards)/( Pass Attempts + Rush Attempts + Times Sacked)',
 'turnovers': 'Team Turnovers Lost',
 'fumbles_lost': 'Fumbles Lost by Team',
 'pass_cmp': 'Passes completed',
 'pass_att': 'Passes attempted',
 'pass_yds': 'Yards Gained by Passing<br>For teams, sack yardage is deducted from this total',
 'pass_td': 'Passing Touchdowns',

In [377]:
th_rows

[<th aria-label="Week number in season" class=" poptip sort_default_asc sorttable_nosort center" data-stat="week_num" data-tip="Week number in season" scope="col">Week</th>,
 <th aria-label="Day" class=" poptip sort_default_asc left" data-stat="game_day_of_week" scope="col">Day</th>,
 <th aria-label="Date" class=" poptip sort_default_asc center" data-stat="game_date" scope="col">Date</th>,
 <th aria-label="Winner/tie" class=" poptip sort_default_asc center" data-stat="winner" scope="col">Winner/tie</th>,
 <th aria-label="game_location" class=" poptip sort_default_asc center" data-stat="game_location" scope="col"></th>,
 <th aria-label="Loser/tie" class=" poptip sort_default_asc center" data-stat="loser" scope="col">Loser/tie</th>,
 <th aria-label="boxscore_word" class=" poptip sort_default_asc center" data-stat="boxscore_word" scope="col"></th>,
 <th aria-label="Points Scored by the winning team (first one listed)" class=" poptip center" data-stat="pts_win" data-tip="Points Scored by t

In [349]:
# all_divs = ['all_AFC', 'all_NFC', 'all_playoff_results', 'all_afc_playoff_standings', 'all_nfc_playoff_standings', 'all_team_stats', 'all_passing', 'all_rushing', 'all_returns', 'all_kicking', 'all_team_scoring', 'all_team_conversions', 'all_drives']
# div_dict = {}
# for divs in all_divs:
    div_val = soup.find('div', {'id':divs})
    table_rows = div_val.find_all('tr')
    if table_rows == []:
        comments = div_val.find_all(string=lambda text: isinstance(text, Comment))
        table = BeautifulSoup(str(comments), 'lxml')
        rows = table.find_all('tr')
        div_dict[divs] = rows
    else:
        div_dict[divs] = table_rows

weeks = soup.find('div', {'id':'all_week_games'})
comments = weeks.find_all(string=lambda text: isinstance(text, Comment))
table = BeautifulSoup(str(comments), 'lxml')
vals = table.find_all('a')
val_dict = {}
for i in vals:
#     val_dict[i.string] = i['href'].split('/')[-1]
    val_dict[i.string] = i['href']
div_dict['all_week_games'] = val_dict

# awards = soup.find('div', {'id':'all_awards'})
# comments = awards.find_all(string=lambda text: isinstance(text, Comment))
# table = BeautifulSoup(str(comments), 'lxml')
# vals = table.find_all('a')
# val_dict = {}
# for i in range(0,len(vals),2):
#     val_dict[vals[i].string] = {'award_id': vals[i]['href'].split('/')[-1].split('.')[0], 'player_name': vals[i+1].string, 'player_id': vals[i+1]['href'].split('/')[-1].split('.')[0]}
# div_dict['all_awards'] = val_dict

In [352]:
awards = soup.find('div', {'id':'all_awards'})
comments = awards.find_all(string=lambda text: isinstance(text, Comment))
table = BeautifulSoup(str(comments), 'lxml')
vals = table.find_all('a')
val_dict = {}
for i in range(0,len(vals),2):
    val_dict[vals[i].string] = {'award_id': vals[i]['href'].split('/')[-1].split('.')[0], 'player_name': vals[i+1].string, 'player_id': vals[i+1]['href'].split('/')[-1].split('.')[0]}

In [353]:
vals

[<a href="/players/award_apmvp.htm">AP MVP</a>,
 <a href="/players/B/BradTo00.htm">Tom Brady</a>,
 <a href="/players/award_pfwamvp.htm">PFWA MVP</a>,
 <a href="/players/B/BradTo00.htm">Tom Brady</a>,
 <a href="/players/award_bba.htm">Bert Bell Award (Player of the Year)</a>,
 <a href="/players/W/WentCa00.htm">Carson Wentz</a>,
 <a href="/players/award_apopoy.htm">AP Offensive Player of the Year</a>,
 <a href="/players/G/GurlTo01.htm">Todd Gurley</a>,
 <a href="/players/award_apdpoy.htm">AP Defensive Player of the Year</a>,
 <a href="/players/D/DonaAa00.htm">Aaron Donald</a>,
 <a href="/players/award_sbmvp.htm">Super Bowl MVP</a>,
 <a href="/players/F/FoleNi00.htm">Nick Foles</a>,
 <a href="/players/award_aporoy.htm">AP Offensive Rookie of the Year</a>,
 <a href="/players/K/KamaAl00.htm">Alvin Kamara</a>,
 <a href="/players/award_apdroy.htm">AP Defensive Rookie of the Year</a>,
 <a href="/players/L/LattMa01.htm">Marshon Lattimore</a>,
 <a href="/players/award_wpmoy.htm">Walter Payton Ma

In [351]:
div_dict['all_awards']

{'AP MVP': {'award_id': 'award_apmvp',
  'player_name': 'Tom Brady',
  'player_id': 'BradTo00'},
 'PFWA MVP': {'award_id': 'award_pfwamvp',
  'player_name': 'Tom Brady',
  'player_id': 'BradTo00'},
 'Bert Bell Award (Player of the Year)': {'award_id': 'award_bba',
  'player_name': 'Carson Wentz',
  'player_id': 'WentCa00'},
 'AP Offensive Player of the Year': {'award_id': 'award_apopoy',
  'player_name': 'Todd Gurley',
  'player_id': 'GurlTo01'},
 'AP Defensive Player of the Year': {'award_id': 'award_apdpoy',
  'player_name': 'Aaron Donald',
  'player_id': 'DonaAa00'},
 'Super Bowl MVP': {'award_id': 'award_sbmvp',
  'player_name': 'Nick Foles',
  'player_id': 'FoleNi00'},
 'AP Offensive Rookie of the Year': {'award_id': 'award_aporoy',
  'player_name': 'Alvin Kamara',
  'player_id': 'KamaAl00'},
 'AP Defensive Rookie of the Year': {'award_id': 'award_apdroy',
  'player_name': 'Marshon Lattimore',
  'player_id': 'LattMa01'},
 'Walter Payton Man of the Year': {'award_id': 'award_wpmoy'

In [163]:
def name_extract(cell):
    cell_id = re.findall('[^a-z]([a-z]{3})[^a-z]',cell['href'])[0]#.split('/')[-1].split('.')[0]
    cell_name = cell.string
    return cell_id, cell_name

def id_name_add(cell, pbp_dict, id_name_str):
    id_str, name_str = name_extract(cell)
    pbp_dict[id_name_str + '_id'] = id_str
    pbp_dict[id_name_str + '_name'] = name_str
    return pbp_dict

In [288]:
stat_list = []
for row in div_dict['all_AFC'][2:]:
    cells = row.contents
    if cells[0].a is not None:
        stat_dict = {}
        for cell in cells:
            lbl = cell['data-stat']
            cell_str = cell.string
            if lbl in ['team']:
                stat_dict = id_name_add(cell.a, stat_dict, lbl)
            else:
                stat_dict[cell['data-stat']] = cell_str
        stat_list.append(stat_dict)
#     break
# 'all_AFC', 'all_NFC'

In [346]:
stat_list = []
for row in div_dict['all_drives'][1:]:
    cells = row.contents
    if cells[0].name is not None and cells[1].a is not None:
        stat_dict = {}
        for cell in cells:
            lbl = cell['data-stat']
            cell_str = cell.string
            if cell_str is None:
                stat_dict[cell['data-stat']] = 0
            elif lbl in ['team']:
                stat_dict = id_name_add(cell.a, stat_dict, lbl)
            elif lbl.find('perc')>-1 or lbl.find('pct')>-1:
                stat_dict[cell['data-stat']] = float(cell_str[:-1])
            elif lbl == 'start_avg':
                stat_dict['yards_from_endzone_avg'] = float(cell['csk'])
            elif lbl == 'time_avg':
                tm = cell_str.split(':')
                stat_dict['sec_avg'] = int(tm[0])*60 + int(tm[1])
            else:
                stat_dict[cell['data-stat']] = float(cell_str)
        stat_list.append(stat_dict)
#     break
# 'all_team_stats', 'all_passing', 'all_rushing', 'all_returns', 'all_kicking', 'all_team_scoring', 'all_team_conversions', 'all_drives'

In [347]:
stat_list

[{'ranker': 1.0,
  'team_id': 'crd',
  'team_name': 'Arizona Cardinals',
  'g': 16.0,
  'drives': 202.0,
  'play_count_tip': 1091.0,
  'score_pct': 29.0,
  'turnover_pct': 12.0,
  'plays_per_drive': 5.4,
  'yds_per_drive': 24.9,
  'yards_from_endzone_avg': 70.3,
  'sec_avg': 148,
  'points_avg': 1.41},
 {'ranker': 2.0,
  'team_id': 'jax',
  'team_name': 'Jacksonville Jaguars',
  'g': 16.0,
  'drives': 199.0,
  'play_count_tip': 1115.0,
  'score_pct': 34.0,
  'turnover_pct': 11.0,
  'plays_per_drive': 5.6,
  'yds_per_drive': 29.4,
  'yards_from_endzone_avg': 70.8,
  'sec_avg': 156,
  'points_avg': 1.82},
 {'ranker': 3.0,
  'team_id': 'nyj',
  'team_name': 'New York Jets',
  'g': 16.0,
  'drives': 199.0,
  'play_count_tip': 1019.0,
  'score_pct': 28.0,
  'turnover_pct': 11.0,
  'plays_per_drive': 5.1,
  'yds_per_drive': 24.5,
  'yards_from_endzone_avg': 72.4,
  'sec_avg': 146,
  'points_avg': 1.5},
 {'ranker': 4.0,
  'team_id': 'sea',
  'team_name': 'Seattle Seahawks',
  'g': 16.0,
  'dr

In [250]:
stat_list = []
for row in div_dict['all_nfc_playoff_standings'][1:]:
    cells = row.contents
    if cells[0].a is not None:
        stat_dict = {}
        for cell in cells:
            lbl = cell['data-stat']
            cell_str = cell.string
            if lbl in ['team']:
                stat_dict = id_name_add(cell.a.next, stat_dict, lbl)
            else:
                stat_dict[cell['data-stat']] = cell_str
        stat_list.append(stat_dict)
        break
# 'all_afc_playoff_standings', 'all_nfc_playoff_standings

In [253]:
cells[0].name

'th'

In [242]:
stat_list

[]

In [360]:
stat_list = []
for row in div_dict['all_playoff_results'][1:]:
    cells = row.contents
    stat_dict = {}
    for cell in cells:
        if cells[0].name is not None:
            lbl = cell['data-stat']
            cell_str = cell.string
            if lbl in ['team', 'loser', 'winner']:
                stat_dict = id_name_add(cell.a, stat_dict, lbl)
            elif lbl in ['game_location']:
                if cell_str is None:
                    stat_dict[cell['data-stat']] = 'winner'
                else:
                    stat_dict[cell['data-stat']] = 'loser'
            elif lbl in ['boxscore_word']:
                stat_dict['boxscore'] = cell.a['href']
            else:
                stat_dict[cell['data-stat']] = cell_str
    stat_list.append(stat_dict)
#     break
            
            
# 'all_playoff_results'

In [361]:
stat_list

[{'week_num': 'WildCard',
  'game_day_of_week': 'Sat',
  'game_date': 'January 6',
  'winner_id': 'atl',
  'winner_name': 'Atlanta Falcons',
  'game_location': 'loser',
  'loser_id': 'ram',
  'loser_name': 'Los Angeles Rams',
  'boxscore': '/boxscores/201801060ram.htm',
  'pts_win': '26',
  'pts_lose': '13'},
 {'week_num': 'WildCard',
  'game_day_of_week': 'Sat',
  'game_date': 'January 6',
  'winner_id': 'oti',
  'winner_name': 'Tennessee Titans',
  'game_location': 'loser',
  'loser_id': 'kan',
  'loser_name': 'Kansas City Chiefs',
  'boxscore': '/boxscores/201801060kan.htm',
  'pts_win': '22',
  'pts_lose': '21'},
 {'week_num': 'WildCard',
  'game_day_of_week': 'Sun',
  'game_date': 'January 7',
  'winner_id': 'jax',
  'winner_name': 'Jacksonville Jaguars',
  'game_location': 'winner',
  'loser_id': 'buf',
  'loser_name': 'Buffalo Bills',
  'boxscore': '/boxscores/201801070jax.htm',
  'pts_win': '10',
  'pts_lose': '3'},
 {'week_num': 'WildCard',
  'game_day_of_week': 'Sun',
  'game

In [162]:
aa = cells[0].a['href']
re.findall('[^a-z]([a-z]{3})[^a-z]',aa)[0]

'den'

In [170]:
pd.DataFrame(stat_list)

""


In [120]:
cells[0].a.string

'New England Patriots'

In [121]:
div_dict['all_AFC'][6].contents[0].a is None

True

In [92]:
tst = soup.find('div', {'id':'all_awards'})
comments = tst.find_all(string=lambda text: isinstance(text, Comment))
table = BeautifulSoup(str(comments), 'lxml')
vals = table.find_all('a')
val_dict = {}
for i in range(0,len(vals),2):
    val_dict[vals[i].string] = {'award_id': vals[i]['href'].split('/')[-1].split('.')[0], 'player_name': vals[i+1].string, 'player_id': vals[i+1]['href'].split('/')[-1].split('.')[0]}

In [93]:
val_dict

{'AP MVP': {'award_id': 'award_apmvp',
  'player_name': 'Tom Brady',
  'player_id': 'BradTo00'},
 'PFWA MVP': {'award_id': 'award_pfwamvp',
  'player_name': 'Tom Brady',
  'player_id': 'BradTo00'},
 'Bert Bell Award (Player of the Year)': {'award_id': 'award_bba',
  'player_name': 'Carson Wentz',
  'player_id': 'WentCa00'},
 'AP Offensive Player of the Year': {'award_id': 'award_apopoy',
  'player_name': 'Todd Gurley',
  'player_id': 'GurlTo01'},
 'AP Defensive Player of the Year': {'award_id': 'award_apdpoy',
  'player_name': 'Aaron Donald',
  'player_id': 'DonaAa00'},
 'Super Bowl MVP': {'award_id': 'award_sbmvp',
  'player_name': 'Nick Foles',
  'player_id': 'FoleNi00'},
 'AP Offensive Rookie of the Year': {'award_id': 'award_aporoy',
  'player_name': 'Alvin Kamara',
  'player_id': 'KamaAl00'},
 'AP Defensive Rookie of the Year': {'award_id': 'award_apdroy',
  'player_name': 'Marshon Lattimore',
  'player_id': 'LattMa01'},
 'Walter Payton Man of the Year': {'award_id': 'award_wpmoy'

In [30]:
tr = soup.find('div', {'id':'all_playoff_results'})
comments = tr.find_all(string=lambda text: isinstance(text, Comment))
table = BeautifulSoup(str(comments), 'lxml')
rows = table.find_all('tr')

In [61]:
div_dict['all_drives'][0]

<tr class="over_header">\n         <th aria-label="" class=" over_header center" colspan="2" data-stat=""></th><th></th>\n         <th aria-label="" class=" over_header center" colspan="4" data-stat=""></th>\n         <th aria-label="" class=" over_header center" colspan="5" data-stat="averages">Average Drive</th>\n      </tr>

In [52]:
div_dict['all_team_stats']

[]

In [11]:
# afc_div = soup.find('div', {'id':'all_AFC'})
# nfc_div = soup.find('div', {'id':'all_NFC'})
# playoff_div = soup.find('div', {'id':'all_playoff_results'})
# all_AFC
# all_NFC
# all_playoff_results
# all_afc_playoff_standings
# all_nfc_playoff_standings
# all_week_games
# all_awards
# all_team_stats
# all_passing
# all_rushing
# all_returns
# all_kicking
# all_team_scoring
# all_team_conversions
# all_drives
playoff_div    

<div class="table_wrapper setup_commented commented" id="all_playoff_results">
<div class="section_heading">
<span class="section_anchor" data-label="Playoff Results" id="playoff_results_link"></span><h2>Playoff Results</h2> <div class="section_heading_text">
<ul>
</ul>
</div>
</div>
<div class="placeholder"></div>
<!--
   <div class="table_outer_container">
      <div class="overthrow table_container" id="div_playoff_results">
  <table class="sortable stats_table" id="playoff_results" data-cols-to-freeze=1><caption>Playoff Results Table</caption>
   <colgroup><col><col><col><col><col><col><col><col><col></colgroup>
   <thead>      
      <tr>
         <th aria-label="Week number in season" data-stat="week_num" scope="col" class=" poptip sort_default_asc sorttable_nosort center" data-tip="Week number in season" >Week</th>
         <th aria-label="Day" data-stat="game_day_of_week" scope="col" class=" poptip sort_default_asc left" >Day</th>
         <th aria-label="Date" data-stat="game_

In [8]:
afc_div

<div class="table_wrapper floated columns" id="all_AFC">
<div class="section_heading">
<span class="section_anchor" data-label="AFC Standings" id="AFC_link"></span><h2>AFC Standings</h2> <div class="section_heading_text">
<ul> <li>* - division winner, + - wild card</li>
</ul>
</div>
</div> <div class="table_outer_container">
<div class="overthrow table_container" id="div_AFC">
<table class="sortable stats_table" data-cols-to-freeze="1" id="AFC"><caption>AFC Standings Table</caption>
<colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup>
<thead>
<tr>
<th aria-label="Tm" class=" poptip sort_default_asc show_partial_when_sorting left" data-stat="team" scope="col">Tm</th>
<th aria-label="Wins" class=" poptip center" data-stat="wins" data-tip="Games Won" scope="col">W</th>
<th aria-label="Losses" class=" poptip center" data-stat="losses" data-tip="Games Lost" scope="col">L</th>
<th aria-label="Win-Loss Percentage of teamFor coaches, minimum to qualify 